Это на случай, если я завтра не проснусь и не доделаю тетрадку. Пока тут просто много плохого кода, но всё работает, я бы сказал, что достаточно неплохо

In [1]:
# -*- coding: utf-8 -*-
# !/usr/bin/env python
# Written by Milind Deore <tomdeore@gmail.com>, March-2020

from __future__ import print_function
import pickle
import os.path
import sys
import random
import wikipediaapi
import uuid
from datetime import date
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

DEBUG = 0
if DEBUG:
    import json

class Covid19Docs:
    def __init__(self):
        # If modifying these scopes, delete the file token.pickle.
        self.DRIVE_SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents']
        self.creds = None
        # Add all your created templates in the below list.
        self.templates = ['1a0C3AQ290uc0yFcVhLD36sj12mQdmv_VnSjh4WhnNvQ']
        self.drive_service = None
        self.docs_service = None
        self.wiki = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
        self.covid19_wiki = self.wiki.page("Coronavirus_disease_2019")


        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.DRIVE_SCOPES)
                self.creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(self.creds, token)

        if self.creds == None:
            print('ERROR : Service credentials unavailable!')
            sys.exit()

        # Start drive and docs services.
        self.drive_service = build('drive', 'v3', credentials=self.creds)
        self.docs_service = build('docs', 'v1', credentials=self.creds)



    def pick_a_template(self):
        """
        There can be a list of templated that you can pick from. Make sure you add them
        to template list.
        """

        title = 'covid-19_' + str(uuid.uuid1())
        body = {
            'name': title,
            'description': 'Covid-19 Wiki'
        }
        drive_response = self.drive_service.files().copy(
            fileId=random.choice(self.templates), body=body).execute()
        document_copy_id = drive_response.get('id')
        print('Dup copy ID : {0}'.format(document_copy_id))

        return document_copy_id


    def replace_a_data(self, key, value):
        """
        Replace text in the document using 'key/tag'
        """
        dic = {'replaceAllText': {
                    'containsText': {
                        'text': '{{' + key + '}}',
                        'matchCase':  'true'
                    },
                    'replaceText': value,
                }}
        return dic


    def insert_bullet_text(self, idx, title):
        """
        Create bullets in the document.
        """

        requests = [
         {
            'insertText': {
                'location': {
                    'index': idx,
                },
                'text': title
            }},{
                'createParagraphBullets': {
                 'range': {
                     'startIndex': idx,
                     'endIndex':  idx + len(title)
                 },
                 'bulletPreset': 'BULLET_DIAMONDX_ARROW3D_SQUARE',
             }
        }]

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        if result:
            pass

        return len(title)


    def insert_text(self, idx, text):
         """
         Insert raw text without formating.
         """
         req = {'insertText': {
                 'location': {
                     'index': idx,
                 },
                 'text': text,
             }}

         return req, len(text)


    def format_text(self, starti, endi, is_bold, is_italic, is_underline):
        """
        Format the text as following:
        - Bold
        - Italic
        """
        req = {'updateTextStyle': {
                'range': {
                    'startIndex': starti,
                    'endIndex': endi
                },
                'textStyle': {
                    'bold': is_bold,
                    'italic': is_italic,
                    'underline': is_underline,
                    'weightedFontFamily': {
                                           'fontFamily': 'Times New Roman'
                                           },
                    'fontSize': {
                                'magnitude': 12,
                                'unit': 'PT'
                                },
                },
                'fields': 'bold, italic, weightedFontFamily, fontSize'
            }}

        return req


    def hwd_batch_update(self, doc_id, requests):
        """
        Batch update the document with the requests.
        """

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result


    def hwd_insert_hyperlink(self, doc_id, start_idx, end_idx, url):
        """
        Insert hyperlink to the text for a given range in the
        document body.
        """

        requests = [
          {
           "updateTextStyle": {
            "textStyle": {
             "link": {
              "url": url
             }
            },
            "range": {
             "startIndex": start_idx,
             "endIndex": end_idx
            },
            "fields": "link"
           }}
        ]

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result


    def get_text_range_idx(self, doc_id, match_text):
        """
        Find text and their start and end index.
        """

        # Do a document "get" request and print the results as formatted JSON
        result = self.docs_service.documents().get(documentId=doc_id).execute()
        if DEBUG:
            print('RX Data {0}'.format(json.dumps(result, indent=4)))
        with open('data.json', 'w') as f:
            json.dump(result, f, indent=4)
        data = result.get('body').get('content')
        startIdx = 0
        endIdx = 0

        for d in data:
            para = d.get('paragraph')
            if para is None:
                continue
            else:
                elements = para.get('elements')
                for e in elements:
                    if e.get('textRun'):
                        content = e.get('textRun').get('content')
                        print(' {}'.format(content))
                        if match_text in content:
                            print('matched')
                            startIdx = e.get('startIndex')
                            endIdx = e.get('endIndex')

        return startIdx, endIdx


    def insert_image(self, start_idx, url, h, w):
        """
        Insert PNG, JPEG, GIF images inline, while adding text.
        """

        request = [{
            'insertInlineImage': {
                'location': {
                    'index': start_idx
                },
                'uri':
                    url,
                'objectSize': {
                    'height': {
                        'magnitude': h,
                        'unit': 'PT'
                    },
                    'width': {
                        'magnitude': w,
                        'unit': 'PT'
                    }
                }
            }
        }]

        return request


    def print_main_sections(self, doc_id, endi):
        section_titles = ['Signs and symptoms', 'Cause', 'Diagnosis', 'Prevention']
        requests = []
        wr_idx = endi + 1

        for s in self.covid19_wiki.sections:
            if s.title in section_titles:
                section_title = '\n' + s.title + ' :\n'
                req, idx = self.insert_text(wr_idx, section_title)
                requests.append(req)
                # req = self.format_text(wr_idx, wr_idx + idx, True, False, False)
                # requests.append(req)
                wr_idx += idx
                req, idx = self.insert_text(wr_idx, s.text + '\n')
                requests.append(req)
                wr_idx += idx

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result, endi

In [2]:
import json

In [ ]:
if __name__ == '__main__':

    c_docs = Covid19Docs()
    # doc_id = c_docs.pick_a_template()

    requests = []
    requests.append(c_docs.replace_a_data('title', 'Coronavirus Disease 2019'))

    today = date.today()
    requests.append(c_docs.replace_a_data('date', '{0}'.format(today)))
    # c_docs.hwd_batch_update(doc_id, requests)

    starti, endi = c_docs.get_text_range_idx(doc_id, 'Details :')
    rlt, endi = c_docs.print_main_sections(doc_id, endi)

In [4]:
title = 'My Document'
body = {
    'title': title
}
doc = c_docs.docs_service.documents() \
    .create(body=body).execute()
print('Created document with title: {0}'.format(
    doc.get('title')))

doc_id = doc.get('documentId')

Created document with title: My Document


In [37]:
requests = [
     {
        'insertText': {
            'location': {
                'index': 1,
            },
            'text': "blablablabla\n"
        }
    },
             {
        'insertText': {
            'location': {
                'index': 2,
            },
            'text': "uuuu"
        }
    },
             {
        'insertText': {
            'location': {
                'index': 3,
            },
            'text': "c"
        }
    },
]

result = c_docs.docs_service.documents().batchUpdate(
    documentId=idx, body={'requests': requests}).execute()

In [ ]:

section_titles = ['Signs and symptoms', 'Cause', 'Diagnosis', 'Prevention']
requests = []
wr_idx = endi + 1

for s in self.covid19_wiki.sections:
    if s.title in section_titles:
        section_title = '\n' + s.title + ' :\n'
        req, idx = self.insert_text(wr_idx, section_title)
        requests.append(req)
        # req = self.format_text(wr_idx, wr_idx + idx, True, False, False)
        # requests.append(req)
        wr_idx += idx
        req, idx = self.insert_text(wr_idx, s.text + '\n')
        requests.append(req)
        wr_idx += idx

result = self.docs_service.documents().batchUpdate(
    documentId=doc_id, body={'requests': requests}).execute()

In [57]:
requests = []
endi = 0
wr_idx = endi + 1

req, idx = c_docs.insert_text(wr_idx, "AAA")
requests.append(req)
wr_idx += idx

req, idx = c_docs.insert_text(wr_idx, "BBB\n")
requests.append(req)
wr_idx += idx

req, idx = c_docs.insert_text(wr_idx, "ССС")
requests.append(req)
wr_idx += idx

result = c_docs.docs_service.documents().batchUpdate(
    documentId=doc_id, body={'requests': requests}).execute()

In [98]:
# loading in modules
import sqlite3
import pandas as pd

# creating file path
dbfile = 'AS2.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)

# Be sure to close the connection
df2 = pd.read_sql_query('SELECT html, lexeme FROM dictionary', con)
con.close()

[('dictionary',), ('for_search',)]


In [3]:
df3

,html,lexeme
0,"<b>ЗЯТЬ</b> , СУЩ; МУЖСК, ОДУШ; <i>-я</i> , М...",ЗЯТЬ
1,"<b>ДА1</b> , ЧАСТ. <b>I</b> , просодически офо...",ДА1
2,"<b>ДА2</b> , СОЮЗ или ЧАСТ. <b>I</b> , СОЮЗ. <...",ДА2
3,"<b>ДАВА́ТЬ</b> , ГЛАГ; <i>даю</i> ́, <i> даёт...",ДАВА́ ТЬ
4,"<b>ДАВА́ТЬСЯ</b> , ГЛАГ; <i>даю</i> ́ <i>сь</...",ДАВА́ ТЬСЯ
...,...,...
1165,"<b>ЗУБР</b> , СУЩ; МУЖСК, ОДУШ; - <i>а</i> , М...",ЗУБР
1166,"<b>ЗУБРИ́ТЬ</b> , ГЛАГ; <i>зубрю</i> ́ <i>, з...",ЗУБРИ́ ТЬ
1167,"<b>ЗУД</b> , СУЩ; МУЖСК; - <i>а</i> . <b>зуд 1...",ЗУД
1168,"<b>ЗЫ́БКИЙ</b> , ПРИЛ; <i>-ая, -ое,</i> КР ...",ЗЫ́ БКИЙ


In [6]:
sample = df3.sample(1).html.values[0]
from bs4 import BeautifulSoup
html = BeautifulSoup(sample)

def acc_strip(s):
    
    accents = {"а́": "а", "е́": "е", "и́": "и",
               "о́": "о", "у́": "у", "ы́": "ы",
               "э́": "э", "ю́": "ю", "я́": "я"}
    
    temp = s.lower()
    
    for accent in accents:
        temp = temp.replace(accent, f"{accents[accent]}^")
        
    return temp.upper() if temp != s else s

# print(html.body.find_all(["b", "i"]))
import regex as re
pattern = r"(<[bui]|\/[bui])>(.*?)<([bui]|\/[bui])>"
opa = re.findall(pattern, "<p><b>БИРЮЗО́ВЫЙ</b>, ПРИЛ; <i>-ая, -ое.</i> </p>", overlapped=True)
opa

[('<b', 'БИРЮЗО́ВЫЙ', '/b'),
 ('/b', ', ПРИЛ; ', 'i'),
 ('<i', '-ая, -ое.', '/i')]

In [10]:
import pandas as pd
from tqdm.auto import tqdm
import urllib.request
import requests
import re
import os
from bs4 import BeautifulSoup

import traceback
import ssl
import sqlite3
from pprint import pprint
from fake_useragent import UserAgent
from fp.fp import FreeProxy
from requests import ConnectTimeout, HTTPError, Timeout, ConnectionError

from tqdm.auto import tqdm

ssl.SSLContext.verify_mode = ssl.VerifyMode.CERT_OPTIONAL

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

session = requests.session()
session.trust_env = False

In [11]:
data = []

for i in tqdm(range(50)):
    url = f"http://sem.ruslang.ru/slovnik.php?act=show&from={i}01&to=-1&criteria=&done=&limit=100"
    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    for i in soup.find_all(["tr"])[1::2]:
        name = i.find("a")
        if name:
            data.append(name.text + "-"+ i.find_all("td")[2].text[1:-1]+"")

  0%|          | 0/50 [00:00<?, ?it/s]

In [114]:
df.iloc[0]

new_html    <p><b>А</b>, СОЮЗ; СОЧИН. </p><p style="font-s...
lexeme                                                     А 
Name: 0, dtype: object

In [112]:
for n in range(10):
    q = acc_strip(df.iloc[n].lexeme).replace("^ ", "").lower().strip()

    print(q)
    for i in data:
        if q+"-" in i:
            print(i)
            break

а
а-ЕУ
абордаж
абордаж-ВА
абрикос
абрикос-ЮА
абрикосовый
абрикосовый-ЮА
абсолютный
абсолютный-ЮА
абстрактный
абстрактный-ЮА
абсурд
абсурд-ЮА
абсурдный
абсурдный-ЮА
авангард
авангард-ЮА
аванс
аванс-ЮА


In [750]:
69?

'алтарь-ЮА'

In [ ]:
import html as h
import regex as re
import time

for i in tqdm(range(df1.shape[0])):
    
    time.sleep(5)
    
    sub_df = df1.iloc[i]
    title = acc_strip(sub_df.lexeme).replace("^ ", "").lower().strip()
    for i in data:
        if title+"-" in i:
            title = i
            break

    requests = []
    endi = 0
    wr_idx = endi + 1

    # c_docs = Covid19Docs()
    body = {
        'title': title
    }
    doc = c_docs.docs_service.documents() \
        .create(body=body).execute()
    # print('Created document with title: {0}'.format(
    #     doc.get('title')))

    doc_id = doc.get('documentId')
    sample = sub_df.new_html
    html = BeautifulSoup(sample)

    d_c = 0
    for section in html.body.find_all("p"):
        pattern = r"(<[buip]|\/[buip])>(.*?)<([buip]|\/[buip])>"
        opa = re.findall(pattern, str(section), overlapped=True)
        b_i = section.find_all(["b", "i"], text=True, recursive=False)
        # bold = section.find("b")
        # italic = section.find_all("i")
        default = section.find_all(text=True, recursive=False)

        line = ""
        # for left, content, right in opa:

            # print(left, content, right)

        # print(line)

        for left, content, right in opa:
            if content == "": continue
            req, idx = c_docs.insert_text(wr_idx, h.unescape(content))
            requests.append(req)
            # if "•" in content:
            #     if d_c != 0:
            #         content = content.replace("•", "\n•")
            #         req, idx = c_docs.insert_text(wr_idx, "\n")
            #         requests.append(req)
            #         wr_idx += idx
            #     else:
            #         d_c += 1
            # print(content)
            # print(content)

            if "b" in left and "/b" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, True, False, False)
                requests.append(req)
            elif "i" in left and "/i" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, True, False)
                requests.append(req)
            elif "u" in left and "/u" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, True)
                requests.append(req)
            else:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, False)
                requests.append(req)
            wr_idx += idx

        req, idx = c_docs.insert_text(wr_idx, "\n")
        requests.append(req)
        wr_idx += idx

        # print("----\n")
        # print(content)

    # requests.append(
    #     {
    #         'updateTextStyle': {
    #             'range': {
    #                 'startIndex': 1,
    #                 'endIndex': wr_idx
    #             },
    #             'textStyle': {
    #                 'weightedFontFamily': {
    #                     'fontFamily': 'Times New Roman'
    #                 },
    #                 'fontSize': {
    #                     'magnitude': 12,
    #                     'unit': 'PT'
    #                 },
    #                     },
    #             'fields': 'weightedFontFamily, fontSize, bold, italic,'
    #                 },
    #             }
    # )


    result = c_docs.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

  0%|          | 0/692 [00:00<?, ?it/s]

In [267]:
c = "ЗНАЧЕНИЕ. ‘Твердые частицы содержащей воду субстанции A1, опустившиеся на дно сосуда, содержащего или содержавшего A1’.УПРАВЛЕНИЕ.А1 • КАКОЙ:  кофейная <пивная> гуща."
for sub in re.findall(r"[A-ZА-Я][A-ZА-Я]+", c):
    c = c.replace(sub, "\n"+sub)
c


'\nЗНАЧЕНИЕ. ‘Твердые частицы содержащей воду субстанции A1, опустившиеся на дно сосуда, содержащего или содержавшего A1’.\nУПРАВЛЕНИЕ.А1 • \nКАКОЙ:  кофейная <пивная> гуща.'

In [451]:
def repl(match, count=[0]):
    x, = count
    count[0] += 1
    if x > 0:
        return f'{str(match.group(0)[0])}%{str(match.group(0)[1:])}'
    return str(match.group(0))

def repl_slash(match, count=[0]):
    x, = count
    count[0] += 1
    if x > 0:
        return f'{str(match.group(0)[1:])}%{str(match.group(0)[0])}'
    return str(match.group(0))

In [418]:
re.sub(r"◊", r"AAA", "◊")

'AAA'

In [496]:
sample = sub_df.html
html = BeautifulSoup(sample)

html = str(html)

# for sub in re.findall(r"[A-ZА-Я]+[0-9]+[^а-яА-Яa-zA-Z\.]*?[A-ZА-Я]+?[\.;:]", html, overlapped=True):
#     html = html.replace(sub, f"%{sub}")
html = re.sub(r"([A-ZА-Я]+[0-9]+[^а-яА-Яa-zA-Z\.]*?[A-ZА-Я]+?[\.;:])", r"%\1", html, flag=1)

    
# for sub in re.findall(r"[^\d] [A-ZА-Я]+[0-9]+\h[\u2022,\u2023,\u25E6,\u2043,\u2219].*[A-ZА-Я]+?[\.;:]", html, overlapped=True):
#     html = html.replace(sub, f"%{sub}")

html = re.sub(r"([^\d]\h)([\u2022\u2023\u25E6\u2043\u2219])(.*[A-ZА-Я]+?)", r"\1%\2\3", html)
# print(html)

html = re.sub(r"([^A-ZА-Я][\.;:\>])\h*([A-ZА-Я][A-ZА-Я]+?[\.;:])", r"\1%\2", html)

html = re.sub("(<b>)[^◊]", repl, html)
html = re.sub("[^◊](</b>)", repl_slash, html)

html = BeautifulSoup(html, 'html.parser')
#     html.div.append(BeautifulSoup('<div>insert1</div>', 'html.parser'))
    
d_c = 0
# for section in html.body.find_all("p"):
section = html

pattern = r"(<[buip]|\/[buip])>(.*?)<([buip]|\/[buip])>"
opa = re.findall(pattern, str(section), overlapped=True)
b_i = section.find_all(["b", "i"], text=True, recursive=False)

section

<html><body><b>%ВАКА́НСИЯ%</b> , СУЩ; ЖЕНСК;  <i>-и.Вакансия лаборанта</i> ;  <i>Открылась вакансия начальника отдела</i> ;  <i>В театре открыто десять вакансий в хоре</i> ;  <i>Среди</i> <i>среднего</i> <i>и</i> <i>младшего</i> <i>медицинского</i> <i>персонала</i> <i>вакансий</i> <i>нет</i> .%ЗНАЧЕНИЕ.  <i>Вакансия А1</i> <i>в А2 </i> ‘Никем не занятая должность А1 в организации А2’.%УПРАВЛЕНИЕ.%А1 • РОД:  <i>вакансия руководителя шахматного кружка</i>  &lt; <i>заведующего лабораторией</i> &gt;.• КАКАЯ:  <i>инженерная &lt;офицерская&gt; вакансия</i> .•  <i>в</i>%ПР:  <i>вакансии в больнице &lt;в турфирме</i> &gt;.•  <i>на</i>%ПР:  <i>вакансии на кафедре</i> . %А2 • РОД:  <i>вакансии центра занятости</i> .%СОЧЕТАЕМОСТЬ.  <i>Наличие &lt;отсутствие&gt; вакансий</i> ;  <i>банк вакансий</i> , нов.  <i>ярмарка вакансий</i> ;  <i>число &lt;количество&gt; вакансий</i> ;  <i>сокращение вакансий</i> ;  <i>соискатель вакансии</i> ,  <i>претендент на вакансию</i> ,  <i>конкурс на замещение ваканс

In [114]:
s = ".А3 • в  ВИН:"
re.sub(r"([А-ЯA-Z][0-9][<a-z> \/]*[а-я\.]*?[<a-z> \/]*)([<a-z> \/]*)([\u2022\u2023\u25E6\u2043\u2219][<a-z> \/а-я]+)[<a-z> \/]*([A-ZА-Я][A-ZА-Я]+?)", r"%\1\2\3\4", s)

'.%А3 • в  ВИН:'

In [732]:
re.sub(r"(([A-ZА-Я]*[\d]+)?\h?[\u2022\u2023\u25E6\u2043\u2219]?\h?[\.:;а-я<a-z>\/]*?\h?[A-ZА-Я][A-ZА-Я]+?[\.;:])", r"%\1", "<i> в гуще толпы.</i>") 

'<i> в гуще толпы.</i>'

In [742]:
re.sub(r"(([A-ZА-Я]*[\d]+)?\h?[\u2022\u2023\u25E6\u2043\u2219]?\h?[\.:;а-я<a-z>\/]{0, 2}?\h?[A-ZА-Я][A-ZА-Я]+?[\.;:])", r"%\1", " воду. ЗНАЧЕНИЕ. ‘Твердые")

' воду. ЗНАЧЕНИЕ. ‘Твердые'

In [48]:
re.findall(r"%(<\/b>[<a-z> \/]*?),")

In [ ]:
import html as h
import regex as re
import time
from string import punctuation
from tqdm.auto import tqdm

requests = []

for i in tqdm(range(df2.shape[0])):
    
    time.sleep(5)
    
    def repl(match, count=[0]):
        x, = count
        count[0] += 1
        if x > 0 and str(match.group(0)[-1]) != " ":
            return f'{str(match.group(0)[:-1])}%{str(match.group(0)[-1])}'
        return str(match.group(0))

    def repl_slash(match, count=[0]):
        x, = count
        count[0] += 1
        if x > 0 and str(match.group(0)[0]) != " ":
            return f'{str(match.group(0)[0])}%{str(match.group(0)[1:])}'
        return str(match.group(0))
    
    # time.sleep(10)
    
    sub_df = df2.iloc[i]
#     title = acc_strip(sub_df.lexeme).replace("^ ", "").lower().strip()
#     for i in data:
#         if title+"-" in i:
#             title = i
#             break

    requests = []
    endi = 0
    wr_idx = endi + 1

#     c_docs = Covid19Docs()
#     body = {
#         'title': "TEST"
#     }
#     doc = c_docs.docs_service.documents() \
#         .create(body=body).execute()
#     # print('Created document with title: {0}'.format(
#     #     doc.get('title')))

#     doc_id = doc.get('documentId')
    sample = sub_df.html
    html = BeautifulSoup(sample)
    
    html = str(html)

    # html = re.sub(r"(([A-ZА-Я]*[\d]+)?\h?[\u2022\u2023\u25E6\u2043\u2219]?\h?[\.:;<a-z>\/]*?[а-я]*?\h?[A-ZА-Я][A-ZА-Я]+?[\.;:])", r"%\1", html)
    
    # html = re.sub(r"([\.]\h?[A-ZА-Я][A-ZА-Я]+?)", r"%\1", html)
    
    
    html = html.replace(f",%", f",")
    html = html.replace(f";%", f",")
    html = re.sub(r"%,", f",", html)
    html = re.sub(r"%;", f";", html)
    html = re.sub(r"% *([\.,;:]|</[a-z]>)", r"\1%", html)
    
    # for sub in re.findall(r"[^\d] [A-ZА-Я]+[0-9]+\h[\u2022,\u2023,\u25E6,\u2043,\u2219].*[A-ZА-Я]+?[\.;:]", html, overlapped=True):
    #     html = html.replace(sub, f"%{sub}")

    # html = re.sub(r"([^\d][\h\.])([\u2022\u2023\u25E6\u2043\u2219])", r"\1%\2", html)
    # print(html)

    html = h.unescape(html)
    html = re.sub(r"([\.!?][<a-z> \/]*)([A-ZА-Я][A-ZА-Я]+?)", r"\1%\2", html)
    html = re.sub(r"([\.!;?][<a-z> \/]*)([\u2022\u2023\u25E6\u2043\u2219][<a-z> \/])[<a-z> \/]*([A-ZА-Я][A-ZА-Я]+?)", r"\1%\2\3", html)
    html = re.sub(r"([А-ЯA-Z][0-9][<a-z> \/]*[а-я\.]*?[<a-z> \/]*)([<a-z> \/]*)([\u2022\u2023\u25E6\u2043\u2219][<a-z> \/а-я]+)[<a-z> \/]*([A-ZА-Я][A-ZА-Я]+?)", r"%\1\2\3\4", html)

    html = re.sub("(<b>)[^◊]", repl, html)
    html = re.sub("[^◊](</b>)", repl_slash, html)
    
    for dots in [" .", " ,", " ;", " :", " >"]:
        html = html.replace(dots, dots[1:])
    
    html = h.unescape(html)
    html = re.sub(r" +", r" ", html)
    html = re.sub(r"%<\/b>[<a-z> \/]*?‘", "</b> ‘", html)
    html = re.sub(r"%<\/b>[<a-z> \/]*?,", "</b>,", html)
    
#     
    html = html.replace("<i> </i>", "")
    html = html.replace("%</b> <b>%", "</b> <b> ")
    # html = html.replace("<i> ", "<i>")
    # html = html.replace(" <i>", "<i>")
    # html = html.replace("</i> ", "</i>")
    # print(html)
    
    html = BeautifulSoup(html, 'html.parser')
#     html.div.append(BeautifulSoup('<div>insert1</div>', 'html.parser'))
    
    d_c = 0
    # for section in html.body.find_all("p"):
    section = html
    pattern = r"(<[buip]|\/[buip])>(.*?)<([buip]|\/[buip])>"
    opa = re.findall(pattern, str(section), overlapped=True)
    b_i = section.find_all(["b", "i"], text=True, recursive=False)
    # bold = section.find("b")
    # italic = section.find_all("i")
    default = section.find_all(text=True, recursive=False)

    line = ""
    # for left, content, right in opa:

        # print(left, content, right)

    # print(line)
    not_first_b = 0
    for left, content, right in opa:
        # print(left, content, right)
        if "Научное изданиеАктивный словАрь русского языкА" in content:
            print("STAPH!")
            break
        else:
            content = re.sub("%+", "%", content)
            content = re.sub(" +", " ", content)
            content = content.replace("%", "\n")
            if content == "" or content == " ":
                continue
            # if content == "": continue
            # if "b" in left and "/b" in right:
            #     if not_first_b:
            #         content = re.sub(r"(.*)", "\n \1 \n", content)
            #     else:
            #         not_first_b += 1
            # print(left, content, right)
            content = h.unescape(content)
            # content = re.sub(r"(.*•*.*)([A-Z]+)", r"\n\1", content)
            # print(content)
            # if content == " ":
            #     content = content.lstrip(" ")
            req, idx = c_docs.insert_text(wr_idx, content)
            requests.append(req)
            # if "•" in content:
            #     if d_c != 0:
            #         content = content.replace("•", "\n•")
            #         req, idx = c_docs.insert_text(wr_idx, "\n")
            #         requests.append(req)
            #         wr_idx += idx
            #     else:
            #         d_c += 1
            # print(content)
            # print(content)

            if "b" in left and "/b" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, True, False, False)
                requests.append(req)
            elif "i" in left and "/i" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, True, False)
                requests.append(req)
            elif "u" in left and "/u" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, True)
                requests.append(req)
            else:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, False)
                requests.append(req)
            wr_idx += idx

    req, idx = c_docs.insert_text(wr_idx, "\n")
    requests.append(req)
    wr_idx += idx

        # print("----\n")
        # print(content)

    # requests.append(
    #     {
    #         'updateTextStyle': {
    #             'range': {
    #                 'startIndex': 1,
    #                 'endIndex': wr_idx
    #             },
    #             'textStyle': {
    #                 'weightedFontFamily': {
    #                     'fontFamily': 'Times New Roman'
    #                 },
    #                 'fontSize': {
    #                     'magnitude': 12,
    #                     'unit': 'PT'
    #                 },
    #                     },
    #             'fields': 'weightedFontFamily, fontSize, bold, italic,'
    #                 },
    #             }
    # )


    result = c_docs.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

  0%|          | 0/1246 [00:00<?, ?it/s]

In [309]:
html

больше (километра), меньше (килограмма)